In [1]:
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import requests
payload = {
    "priceDate.month": 6,
    "priceDate.day": 17,
    "priceDate.year": 2025,
    "submit": "submit"
}

url = "https://www.treasurydirect.gov/GA-FI/FedInvest/selectSecurityPriceDate"

try:
    response = requests.post(url=url, data=payload, timeout=30)
    response.raise_for_status()

except requests.exceptions.RequestException as e:
    print(f"Error occured: {e}")

In [2]:
output = BeautifulSoup(response.text, "html.parser")
table = output.find("table", class_="data1") # Getting table from webpage

header_row = table.find_all("th")
headers = [th.get_text(strip=True) for th in header_row]

In [9]:
data_rows = table.find_all("tr")[1:]
all_rows = list()
for tr in data_rows:
    row_data = [td.get_text(strip=True) for td in tr.find_all("td")]
    all_rows.append(row_data)

df = pd.DataFrame(data=all_rows, columns=headers)

In [10]:
df

,CUSIP,SECURITY TYPE,RATE,MATURITY DATE,CALL DATE,BUY,SELL,END OF DAY
0,912797NV5,MARKET BASED BILL,0.000%,06/20/2025,,0.000000,99.964750,0.000000
1,912797PT8,MARKET BASED BILL,0.000%,06/24/2025,,99.919500,99.919111,0.000000
2,912797NW3,MARKET BASED BILL,0.000%,06/26/2025,,99.896500,99.896000,0.000000
3,912797PU5,MARKET BASED BILL,0.000%,07/01/2025,,99.838417,99.837833,0.000000
4,912797NX1,MARKET BASED BILL,0.000%,07/03/2025,,99.815556,99.815111,0.000000
...,...,...,...,...,...,...,...,...
453,91282CKM2,MARKET BASED FRN,4.497%,04/30/2026,,100.063279,100.037539,0.000000
454,91282CLA7,MARKET BASED FRN,4.529%,07/31/2026,,100.095285,100.076009,0.000000
455,91282CLT6,MARKET BASED FRN,4.553%,10/31/2026,,100.137565,100.114073,0.000000
456,91282CMJ7,MARKET BASED FRN,4.444%,01/31/2027,,99.981891,99.942419,0.000000
